In [1]:
import os
import re
import json
import pandas as pd
from neo4j import GraphDatabase
import logging

In [2]:
# Neo4j connection class
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def query(self, query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return [record for record in result]

# Initialize the Neo4j database connection
neo4j_conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", password="neo4j")

# Clear existing data
neo4j_conn.query("MATCH (n) DETACH DELETE n")

[]

In [3]:
# Logging setup
logging.basicConfig(filename='graph_import.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [4]:
# JSON processing
json_dir_path = r'C:/Users/jhyang/OneDrive/文档/GitHub_Projects/GraphRAG-Company-Info-Collector/resources/form10k'
max_files_to_import = 500
imported_files_count = 0

In [5]:
# Process JSON files
for json_file in os.listdir(json_dir_path):
    if json_file.endswith('.json') and imported_files_count < max_files_to_import:
        try:
            with open(os.path.join(json_dir_path, json_file), 'r', encoding='utf-8') as file:
                data = json.load(file)

            # Extract company name using 'item1' or a fallback mechanism
            company_name = "Unknown Company"
            if "item1" in data:
                match = re.search(r'the term “Company” means (.+?) and its subsidiaries', data["item1"], re.IGNORECASE)
                if match:
                    company_name = match.group(1)

            # Create or update the Company node
            neo4j_conn.query(
                """
                MERGE (c:Company {name: $company_name})
                RETURN c
                """,
                parameters={"company_name": company_name}
            )

            # Process remaining JSON content
            for key, value in data.items():
                if key != "item1":  # Skip 'item1' as it was used for the company name
                    node_id = f"{company_name}-{key}"  # Unique ID for Content node
                    content_summary = value if isinstance(value, str) else json.dumps(value)[:500]  # Summarize content

                    # Create a Content node
                    neo4j_conn.query(
                        """
                        MERGE (n:Content {id: $node_id})
                        SET n.key = $key, n.summary = $summary
                        RETURN n
                        """,
                        parameters={
                            "node_id": node_id,
                            "key": key,
                            "summary": content_summary
                        }
                    )

                    # Create a relationship between Company and Content
                    neo4j_conn.query(
                        """
                        MATCH (c:Company {name: $company_name})
                        MATCH (n:Content {id: $node_id})
                        MERGE (c)-[:HAS_CONTENT]->(n)
                        """,
                        parameters={
                            "company_name": company_name,
                            "node_id": node_id
                        }
                    )

            # Log success and increment the count
            imported_files_count += 1
            logging.info(f"Processed JSON file: {json_file} for company {company_name}")

        except Exception as e:
            logging.error(f"Error processing file {json_file}: {e}")

        # Stop if the maximum file limit is reached
        if imported_files_count >= max_files_to_import:
            break

In [6]:
# Excel processing
excel_file_path = r'C:/Users/jhyang/OneDrive/文档/GitHub_Projects/GraphRAG-Company-Info-Collector/resources/form13.csv'
df_excel = pd.read_csv(excel_file_path, nrows=500)

In [7]:
for _, row in df_excel.iterrows():
    company_name = row['companyName']  # Company name
    cusip6 = row['cusip6']  # CUSIP6 code
    cusip = row['cusip']  # Full CUSIP code

    # Check if Company exists, update or create
    existing_company = neo4j_conn.query(
        """
        MATCH (c:Company {name: $company_name})
        RETURN c
        """,
        parameters={"company_name": company_name}
    )

    if existing_company:
        neo4j_conn.query(
            """
            MATCH (c:Company {name: $company_name})
            SET c.cusip6 = COALESCE(c.cusip6, $cusip6),
                c.cusip = COALESCE(c.cusip, $cusip)
            RETURN c
            """,
            parameters={"company_name": company_name, "cusip6": cusip6, "cusip": cusip}
        )
        logging.info(f"Updated existing Company node: {company_name}")
    else:
        neo4j_conn.query(
            """
            CREATE (c:Company {name: $company_name, cusip6: $cusip6, cusip: $cusip})
            RETURN c
            """,
            parameters={"company_name": company_name, "cusip6": cusip6, "cusip": cusip}
        )
        logging.info(f"Created new Company node: {company_name}")

In [8]:
# Process CSV to create Manager and Company nodes with relationships
for _, row in df_excel.iterrows():
    manager_name = row['managerName']
    manager_cik = row['managerCik']
    manager_address = row['managerAddress']
    
    company_name = row['companyName']
    cusip6 = row['cusip6']
    cusip = row['cusip']
    
    value = row['value']
    shares = row['shares']
    report_calendar_or_quarter = row['reportCalendarOrQuarter']
    
    # Create or update Manager node
    neo4j_conn.query(
        """
        MERGE (m:Manager {name: $manager_name})
        SET m.cik = $manager_cik, m.address = $manager_address
        RETURN m
        """,
        parameters={
            "manager_name": manager_name,
            "manager_cik": manager_cik,
            "manager_address": manager_address
        }
    )
    
    # Create or update Company node using cusip6 as the linking field
    neo4j_conn.query(
        """
        MERGE (c:Company {cusip: $cusip})
        SET c.name = $company_name, c.cusip = $cusip
        RETURN c
        """,
        parameters={
            "company_name": company_name,
            "cusip6": cusip6,
            "cusip": cusip
        }
    )
    
    # Create HOLDS relationship between Manager and Company with properties
    neo4j_conn.query(
        """
        MATCH (m:Manager {name: $manager_name})
        MATCH (c:Company {cusip: $cusip})
        MERGE (m)-[r:HOLDS]->(c)
        SET r.value = $value, r.shares = $shares, r.reportCalendarOrQuarter = $report_calendar_or_quarter
        RETURN r
        """,
        parameters={
            "manager_name": manager_name,
            "cusip": cusip,
            "value": value,
            "shares": shares,
            "report_calendar_or_quarter": report_calendar_or_quarter
        }
    )


In [9]:
# Close the database connection
neo4j_conn.close()